In [1]:
# -*- coding:utf-8 -*-
import torch
import math
import torch.nn as nn
from torch.autograd import Variable
import os
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
import cv2
from PIL import Image
import numpy as np
import torch.nn.functional as F
from models import *
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
from torch import optim
import torchvision.transforms as transforms
import torch.nn.functional as F
from tqdm import tqdm
import os
import math

In [2]:
class SegDataset(Dataset):
    def __init__(self, root=r'data', w=144, h=144, transform=None,is_train=0):
        file = os.listdir(root)

        imgs = []
        # 每张图
        for line in file:
            imgs.append((os.path.join(root, line, 'img.png'), os.path.join(root, line, 'label.png')))
        # 对于特定类别的，在加一次
        if is_train:
            for line in file:
                if  line == '20220623163957' or line == '20220623164000' or line == '20220623164006':
                    imgs.append((os.path.join(root, line, 'img.png'), os.path.join(root, line, 'label.png')))
            for line in file:
                if  line == '20220623164013' or line == '20220623164024' or line == '20220623164006':
                    imgs.append((os.path.join(root, line, 'img.png'), os.path.join(root, line, 'label.png')))
            for line in file:
                if line == '20220623164013':
                    imgs.append((os.path.join(root, line, 'img.png'), os.path.join(root, line, 'label.png')))
        self.imgs = imgs
        self.transform = transform
        self.width = w
        self.height = h

    def __getitem__(self, index):
        imagename, labelname = self.imgs[index]
        img = Image.open(imagename)
        img = img.resize((self.width, self.height))

        if self.transform is not None:
            img = self.transform(img)

        mask = Image.open(labelname)
        mask = mask.resize((self.width, self.height))
        target = np.array(mask).astype('int32')
        target[target == 255] = -1
        return img, torch.from_numpy(target).long()

    def __len__(self):
        return len(self.imgs)

In [3]:
def train(train_loader, net, optimizer,scheduler, loss_fun,device):
    # set model to train mode
    net.train()

    train_loss = 0
    train_acc = 0
    for inputs, labels in (train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        out = net(inputs)
        optimizer.zero_grad()
        # print(out.shape,labels.shape)
        loss = loss_fun(out, labels)
        train_loss += loss.item()/len(labels)
        
        pred = torch.max(out, 1)[1]
        train_correct = (pred == labels).sum() / (labels.shape[-1] * labels.shape[-2])

        train_acc += train_correct.item()/len(labels)
        
        loss.backward()
        optimizer.step()
    scheduler.step() #每个epoch后的所有batch完成，更新学习率
    torch.save(net.state_dict(),  'net.pth')
    return train_acc / len(train_loader) ,train_loss / math.ceil(len(train_loader))

In [4]:
def test(test_loader,loss_func, net,device):
    # set model to eval mode
    net.eval()
    eval_loss = 0
    eval_acc = 0
    with torch.no_grad():
        for inputs, labels in (test_loader):
            
            inputs = inputs.to(device)
            labels = labels.to(device)

            out = net(inputs)
            loss = loss_func(out, labels)
            eval_loss += loss.item()/len(labels)
            pred = torch.max(out, 1)[1]
            num_correct = (pred == labels).sum() /  (labels.shape[-1] * labels.shape[-2])
            eval_acc += num_correct.item()/len(labels)
            
    return eval_acc / (len(test_loader)),eval_loss / math.ceil(len(test_loader) )

In [5]:
model=Unet(3,4)
# model=FCN32s(4)
# model=Net_seg(4)
net = model.cuda()
device="cuda"
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [10, 20], 0.1)#调整学习率
loss = nn.CrossEntropyLoss(ignore_index=-1)
width = 500
height = 500
batch_size=3

In [6]:
train_transforms = transforms.Compose([

    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

])

train_data = SegDataset(root=r'data', w=width, h=height, transform=train_transforms,is_train=1)
val_data = SegDataset(root=r'data', w=width, h=height, transform=train_transforms)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=val_data, batch_size=batch_size)

In [7]:
train_acc=[]
test_acc=[]
train_loss=[]
test_loss=[]
epoch=1000
max_acc=0
for i in (range(epoch)):
    temp_acc,temp_loss=train(train_loader, net, optimizer,scheduler, loss,device)
    train_acc.append(temp_acc)
    train_loss.append(temp_loss)

    temp_acc,temp_loss=test(test_loader,loss, net,device)
    test_acc.append(temp_acc)
    test_loss.append(temp_loss)
    if temp_acc>=max_acc:
        max_acc=temp_acc
        torch.save(model.state_dict(),  'best.pth')
    print("Epoch: %d  Train Loss: %.3f, Acc: %.3f"%(i+1,train_loss[i],train_acc[i]))
    print("Epoch: %d  Test Loss: %.3f, Acc: %.3f\n"%(i+1,test_loss[i],test_acc[i]))

KeyboardInterrupt: 